# Quantum Matcha TEA library

In [1]:
import numpy as np
from tqdm import trange

import qtealeaves.observables as obs
from qmatchatea.qk_utils import qiskit_get_statevect
from qmatchatea import QCConvergenceParameters, QCOperators, QCBackend
from qmatchatea.py_emulator import run_py_simulation

from qiskit import QuantumCircuit, assemble, transpile, QuantumRegister, AncillaRegister, Aer
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector

from dQA_circuit import HammingEvolution

from lib.dQA_loss import *

In [2]:
N_csi, N = 5, 7

x = np.random.randint(2, size = (N_csi, N) )
x[ x == 0 ] = -1  # data is encoded as +- 1

csi_patterns = x
csi_patterns.shape

(5, 7)

## Test 

In [3]:
# circuit generator
qc_generator = HammingEvolution(num_data_qubits=N)

# actual circuit (in initial superposition)
qc = qc_generator.init_state_plus()

P = 100
dt = 1

for pp in trange(P):
    s_p = (pp+1)/P
    gamma_p = s_p*dt
    beta_p = (1-s_p)*dt

    for mu in range( N_csi ):
    
        # create Hamming error counter circuit based on the given pattern
        qc_counter = qc_generator.Hamming_count(train_data=csi_patterns[mu,:])
        qc_counter_inverse = qc_counter.inverse()
    
        # create evolution circuit
        qc_Uz = qc_generator.U_z(train_data=csi_patterns[mu,:], gamma=gamma_p)
    
        qc = qc.compose(qc_counter)
        qc = qc.compose(qc_Uz)
        qc = qc.compose(qc_counter_inverse)

    # apply Ux
    qc_Ux = qc_generator.U_x(beta_p)
    qc = qc.compose(qc_Ux)

100%|██████████| 100/100 [08:44<00:00,  5.24s/it]


In [4]:
operators = QCOperators()
sigma_z = np.array([[1, 0], [0, -1]])
operators.ops["sz"] = sigma_z

observables = obs.TNObservables()
observables += obs.TNObsBondEntropy()
observables += obs.TNState2File("state.txt", "F")
observables += obs.TNObsLocal('label', 'sz')


conv_params = QCConvergenceParameters(max_bond_dimension=10, singval_mode="C")
backend = QCBackend(backend="PY")

In [5]:
res = run_py_simulation(
    qc,
    convergence_parameters=conv_params,
    operators=operators,
    observables=observables,
    backend=backend
)

In [6]:
res.observables.keys()

dict_keys(['mps_state', 'time', 'energy', 'norm', 'label', 'projective_measurements', 'bond_entropy'])

In [7]:
for i in res.observables["mps_state"]:
    print(i.shape)

(1, 2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 8)
(8, 2, 4)
(4, 2, 2)
(2, 2, 1)


In [8]:
res.observables["bond_entropy"]

{(0, 1): 0.40292107425219215,
 (1, 2): 0.4032721465618253,
 (2, 3): 0.6759355431414515,
 (3, 4): 0.6020741552080249,
 (4, 5): 0.6200753561471024,
 (5, 6): 0.6201507183908781,
 (6, 7): 0.04716626318978098,
 (7, 8): 0.0338491681955336,
 (8, 9): 0.024499660074610675}

In [14]:


obj = mydQA_ancilla(csi_patterns[:,::-1], P, dt, max_bond=10, n_ancilla=3)
obj.init_fourier()
obj.compute_loss( res.observables["mps_state"] )

Array(0.00244564+1.4901161e-08j, dtype=complex64)